In [1]:
### availibility count

In [2]:
import pandas as pd
import os
import numpy as np
import string

In [21]:
def search_homes(filename):
    """
    Method that searches through the Data_dump dict structure
    
    Returns:
        vals: dict where keys include keyword and values are counts
        home_count: # of unique homes where the keyword is present
    """
    lennar = pd.read_json(filename)
    clean = lennar.drop_duplicates(subset=['Availability', 'Price', 'Beds', 'Baths', 'Sqft', 'Address', 'Community', 'URL'],
                    keep='first',
                    ignore_index=True)
    
    
    return clean.groupby('Availability').size().to_dict()

In [22]:
t = search_homes('./data-06-2023/north-carolina/raleigh-final.json')

In [23]:
t

{'Coming soon': 36,
 'Future release': 86,
 'Move-in ready': 10,
 'Under construction': 45}

In [35]:
def market_analysis(date):
    json_files = []
    path = f'./data-{date}/'
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.endswith('.json'):
                json_files.append(root+'/'+name)
    
    cols = ['Coming soon', 'Future release', 'Move-in ready', 'Under construction']
    df = pd.DataFrame([], columns = ['county', 'state']+cols)
    for jf in json_files:
        tmp = pd.read_json(jf)
        state = jf.split('/')[2]
        county = jf.split('/')[3].split('-final')[0]
        d = search_homes(jf)

        for c in cols:
            if c not in d:
                d[c] = 0
        
        if '.ipynb_checkpoints' in county:
            continue
        
        df.loc[len(df)] = [county, state, d['Coming soon'], d['Future release'], 
                          d['Move-in ready'], d['Under construction']]
    
    return df

In [37]:
df = market_analysis('06-2023')

In [38]:
df

,county,state,Coming soon,Future release,Move-in ready,Under construction
0,raleigh,north-carolina,36,86,10,45
1,wilmington,north-carolina,1,40,39,30
2,charlotte,north-carolina,1,97,59,85
3,greensboro,north-carolina,0,0,3,0
4,chicago,illinois,17,58,17,93
...,...,...,...,...,...,...
65,san-francisco-metro-area,california,0,5,7,0
66,san-francisco-bay-area,california,0,106,31,19
67,la-orange-county,california,0,155,18,48
68,,california,0,155,18,48


In [44]:
df.to_csv('./analysis/availability-by-county.csv', index=False)

In [45]:
df2 = df.groupby('state').sum()

In [46]:
df2.to_csv('./analysis/availability-by-state.csv')